
![Image Description](Operations_Research_Methodology_Diagram.png)






In [6]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, listSolvers

# Sample Data
aircraft_data = pd.DataFrame({
    'Aircraft': ['A1', 'A2', 'A3'],
    'MaintenanceTime': [5, 3, 4],
    'RequiredSkill': ['Engine', 'Avionics', 'Engine']
})

technician_data = pd.DataFrame({
    'Technician': ['T1', 'T2', 'T3'],
    'Availability': [8, 8, 8],
    'Skills': [['Engine', 'Avionics'], ['Engine'], ['Avionics']]
})

# Create the optimization problem
model = LpProblem(name="maintenance-scheduling", sense=LpMaximize)

# Decision variables
x = {(t, a): LpVariable(name=f"x_{t}_{a}", cat='Binary') for t in technician_data['Technician'] for a in aircraft_data['Aircraft']}

# Objective function: maximize total maintenance tasks assigned
model += lpSum(x[t, a] for t in technician_data['Technician'] for a in aircraft_data['Aircraft'])

# Constraints
# Each maintenance task should be assigned to exactly one technician
for a in aircraft_data['Aircraft']:
    model += lpSum(x[t, a] for t in technician_data['Technician']) == 1

# Technicians should not exceed their availability
for t in technician_data['Technician']:
    model += lpSum(x[t, a] * aircraft_data.loc[aircraft_data['Aircraft'] == a, 'MaintenanceTime'].values[0] for a in aircraft_data['Aircraft']) <= technician_data.loc[technician_data['Technician'] == t, 'Availability'].values[0]

# Technicians should have the required skills
for t in technician_data['Technician']:
    for a in aircraft_data['Aircraft']:
        required_skill = aircraft_data.loc[aircraft_data['Aircraft'] == a, 'RequiredSkill'].values[0]
        if required_skill not in technician_data.loc[technician_data['Technician'] == t, 'Skills'].values[0]:
            model += x[t, a] == 0

# Solve the problem
status = model.solve()

# Print the results
if LpStatus[status] == 'Optimal':
    print("Optimal Solution Found")
    for t in technician_data['Technician']:
        for a in aircraft_data['Aircraft']:
            if x[t, a].value() == 1:
                print(f"Technician {t} is assigned to aircraft {a}")
else:
    print("No optimal solution found")


Optimal Solution Found
Technician T1 is assigned to aircraft A3
Technician T2 is assigned to aircraft A1
Technician T3 is assigned to aircraft A2


I've been working on a workforce scheduling problem using linear programming in Python with the PuLP library. I'm trying to assign employees to different brands, ensuring certain conditions are met. Here's a detailed explanation of the problem and the code I've implemented:

Problem Statement:
Assign a fixed number of employees to different brands.
Ensure that an employee works on only one brand per day.
An employee can work a maximum of 9 hours and a minimum of 5 hours consecutively.
Need to fulfill staffing requirements for each brand.

In [7]:
import itertools

from pulp import LpProblem, LpVariable, LpBinary, lpSum

n_weeks = 2
days_per_week = 7
total_days = n_weeks * days_per_week
shop_open = 10  # 10 AM
shop_close = 20  # 8 PM
min_working_hours = 5
max_working_hours = 9
n_employees = 25

### DATA

# 5-day or 4-day contract type (60% each for now)
five_days_count = n_employees * 0.6

brands = {
    "PHONE": ["PHONE1", "PHONE2"],
    "TV": ["TV1", "TV2", "TV3"]
}
brand_items = tuple(itertools.chain(*brands.values()))

# employee table  e# : ( 4/5 day , [brand_item, ...] )
employees = {f'e{i}': (5*n_weeks if i < five_days_count else 4*n_weeks, [brand_items[i % len(brand_items)]]) for i in range(n_employees)}
# print(employees)

staffing_requirements = {
    "PHONE1": 1,
    "PHONE2": 1,
    "TV1": 1,
    "TV2": 1,
    "TV3": 1}

# the hours on which it is possible to commence a 5-hr shift
poss_starts = [hr for hr in range(shop_open, shop_close - min_working_hours + 1)]

prob = LpProblem('shift_sked')

### SETS / INDEXES

hours = list(range(shop_open, shop_close))  # convenience for readability...
days = list(range(total_days))
#                                                     Domain trim-down:  vvvv                 vvvv
EDHB_employees = {(e, d, h, b) for e in employees for d in days for h in poss_starts for b in employees[e][1]}
EDHB_brand_items = {(e, d, h, b) for e in employees for d in days for h in hours for b in employees[e][1]}

### VARS

start_shift = LpVariable.dicts('start', indices=EDHB_employees, cat=LpBinary) # e starts shift on day d, hour h, for brand item b
covers = LpVariable.dicts('covers', indices=EDHB_brand_items, cat=LpBinary)  # e covers brand item b on day d, hour h
max_shifts = LpVariable('max_shifts')  # the max shifts by any employee

### OBJ:  Minimize shift starts
prob += lpSum(start_shift)

# alternate objectives for experimenting...
# prob += lpSum(covers)  # should produce days*hours*requirements
# prob += max_shifts  # the max number of shifts by any employee  NOTE:  This is tougher solve, longer...

### CONSTRAINTS

# 1 & 2.  Limit shift starts by 4/5 day limit, and can only start 1 shift/day
for e in employees:
    prob += sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1]) <= employees[e][0]
    for d in days:
        prob += sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1]) <= 1

# 3. Link coverage to shift starting
for (e, d, h, b) in EDHB_brand_items:
    if b not in employees[e][1]:  # this employee cannot 'cover' this item
        prob += covers[e, d, h, b] <= 0
    elif h in poss_starts:  # a start or continued coverage can work...
        prev_hour = covers[e, d, h-1, b] if h > shop_open else None
        prob += covers[e, d, h, b] <= start_shift[e, d, h, b] + prev_hour
    else:  # only previous hour coverage can work (start not possible)
        prob += covers[e, d, h, b] <= covers[e, d, h-1, b]

# 4.  min/max coverage
for e in employees:
    for d in days:
        starts_shift = sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1])
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) <= max_working_hours * starts_shift
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) >= min_working_hours * starts_shift

# 5.  brand-item coverage minimums
for (d, h, b) in itertools.product(days, hours, brand_items):
    prob += sum(covers[e, d, h, b] for e in employees if b in employees[e][1]) >= staffing_requirements[b]

# 6.  Capture max shifts
for e in employees:
    prob += max_shifts >= sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1])

# print(prob)

cbc_path = '/opt/homebrew/opt/cbc/bin/cbc'
solver = pulp.COIN_CMD(path=cbc_path)
res = prob.solve(solver)

# highs_path = '/opt/homebrew/bin/highs'
# solver_2 = pulp.HiGHS_CMD(path=highs_path)
# res = prob.solve(solver_2)

NameError: name 'pulp' is not defined

A markdown

In [ ]:
# TODO @alvaro-cabrera
# 1.  Add a constraint to limit the number of shifts for any employee to 55 

